# 5.6. Regresja logistyczna wielu zmiennych

Wiemy już jak zmierzyć jakość modelu klasyfikacji binarnej. Powróćmy więc do przykładu klasyfikacji ryzyka niewypłacalności klientów. Postaramy się zbudować model regresji logistycznej, który będzie szacował takie ryzyko.

Zmierzymy wszystkie wspomniane w poprzednim rozdziale metryki, chociaż niektóre z nich są dla nas bardziej istotne.

In [1]:
import pandas as pd

In [2]:
credit_cards_df = pd.read_parquet("../data/credit-cards-preprocessed.parquet")
credit_cards_df.sample(n=5).T

ID,29404,18528,20810,24542,23469
LIMIT_BAL,130000.000000,40000.000000,30000.000000,350000.000000,50000.000000
AGE,29.000000,26.000000,49.000000,35.000000,39.000000
PAY_1,0.000000,0.000000,0.000000,0.000000,1.000000
PAY_2,0.000000,0.000000,0.000000,0.000000,2.000000
PAY_3,0.000000,2.000000,0.000000,0.000000,2.000000
PAY_4,0.000000,0.000000,0.000000,0.000000,2.000000
PAY_5,0.000000,0.000000,0.000000,0.000000,2.000000
PAY_6,0.000000,0.000000,0.000000,0.000000,2.000000
BILL_AMT1,58299.000000,37423.000000,29429.000000,218605.000000,16983.000000
BILL_AMT2,59352.000000,40742.000000,29959.000000,166310.000000,16413.000000


## Definicja łańcucha przetwarzania

Po raz kolejny skorzystamy oczywiście z implementacji regresji logistycznej jaką dostarcza scikit-learn. Oczywiście, zgodnie ze sztuką, podzielimy nasz zbiór na część treningową oraz testową, by sprawdzić jego skuteczność na zupełnie nowych danych. Co więcej, przeprowadzimy proces walidacji krzyżowej, która pozwoli nam upewnić się, że obiecujące wyniki nie są dziełem przypadkowego podziału.

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

In [4]:
X = credit_cards_df.drop(columns="DEFAULT")
y = credit_cards_df["DEFAULT"]

In [5]:
lr = LogisticRegression(penalty="none", 
                        max_iter=10000)
scores = cross_validate(
    lr, X, y, cv=10, return_train_score=True,
    scoring=("accuracy", "precision", "recall", "f1")
)
scores

C:\Users\lukaw\anaconda3\envs\intro-to-ml\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukaw\anaconda3\envs\intro-to-ml\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukaw\anaconda3\envs\intro-to-ml\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lukaw\anaconda3\envs\intro-to-ml\lib\site-packages\sklearn

{'fit_time': array([0.61371589, 0.7595017 , 0.63031316, 0.53780007, 0.51923466,
        0.60479331, 0.73248506, 0.57719588, 0.59097147, 0.59785509]),
 'score_time': array([0.01196814, 0.01396275, 0.00897598, 0.0079782 , 0.00897574,
        0.01296568, 0.014009  , 0.00997162, 0.00797629, 0.0079782 ]),
 'test_accuracy': array([0.77833333, 0.77866667, 0.77866667, 0.77866667, 0.77866667,
        0.77866667, 0.779     , 0.779     , 0.779     , 0.779     ]),
 'train_accuracy': array([0.77877778, 0.77877778, 0.77874074, 0.77874074, 0.77881481,
        0.77874074, 0.7787037 , 0.7787037 , 0.77866667, 0.77866667]),
 'test_precision': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'train_precision': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'test_recall': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'train_recall': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'test_f1': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'train_f1': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

Doszliśmy do sytuacji, której obawialiśmy się od początku. Accuracy zwrócone przez stworzony model jest całkiem spore, jednak obie metryki - precision oraz recall - są za każdym razem równe 0. Oznacza to, iż nasz system nie jest w stanie poprawnie klasyfikować przykładów pozytywnych, a najprawdopodobniej zwraca zazwyczaj klasę negatywną.

## Poszukiwanie podprzestrzeni cech o większej skuteczności

Być może nasz model zbyt dobrze dopasowuje się do szumu w danych i przez to traci możliwość generalizacji. W związku z tym postarajmy się odnaleźć taki zestaw cech, który trochę zwiększy możliwości naszego modelu.

Rekursywna eliminacja cech jest metodą, która pozwala na odrzucanie kolejnych kolumn ze zbioru danych i przez to znalezienie najlepszego ich zestawu.

In [6]:
from sklearn.feature_selection import RFECV

In [7]:
selector = RFECV(lr, step=1, min_features_to_select=1, 
                 cv=10, scoring="f1")
selector.fit(X, y)

RFECV(cv=10,
      estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                   fit_intercept=True, intercept_scaling=1,
                                   l1_ratio=None, max_iter=10000,
                                   multi_class='auto', n_jobs=None,
                                   penalty='none', random_state=None,
                                   solver='lbfgs', tol=0.0001, verbose=0,
                                   warm_start=False),
      min_features_to_select=1, n_jobs=None, scoring='f1', step=1, verbose=0)

In [8]:
selected_features_df = pd.DataFrame({
    "selected": selector.support_,
    "ranking": selector.ranking_
}, index=X.columns)
selected_features_df

,selected,ranking
LIMIT_BAL,False,8
AGE,True,1
PAY_1,True,1
PAY_2,True,1
PAY_3,True,1
PAY_4,True,1
PAY_5,True,1
PAY_6,True,1
BILL_AMT1,False,4
BILL_AMT2,False,5


Kilka z naszych cech zostało wyrzuconych ze zbioru, co prawdopodobnie pozwoliło osiągnąć lepsze wyniki dla metryki $ F1 $. Sprawdźmy jaka była największa jej wartość.

In [9]:
import numpy as np

In [10]:
np.max(selector.grid_scores_)

0.4040243310199328

Okazuje się, że jesteśmy w stanie wyciągnąć z naszego modelu trochę więcej niż z pełnym zestawem cech. Warto przeprowadzić taki proces, jeśli wyniki naszego modelu są trochę poniżej oczekiwań.

Na użytek kolejnych rozdziałów zapiszmy sobie informację o cechach, które model uznał za przydatne. Nie będziemy gromadzić samej informacji o wybranych cechach, ale zapiszemy zredukowany zbiór.

In [11]:
feature_names = selected_features_df[
    selected_features_df["selected"] == True
].index
feature_names

Index(['AGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
       'PAY_OVERDUE_COUNT', 'WEIGHTED_PAYMENT_HISTORY', 'AVG_PAY_AMT', 'SEX_2',
       'EDUCATION_1', 'EDUCATION_2', 'EDUCATION_3', 'EDUCATION_4',
       'EDUCATION_5', 'MARRIAGE_1', 'MARRIAGE_2', 'MARRIAGE_3'],
      dtype='object')

In [12]:
credit_cards_df[feature_names.tolist() + ["DEFAULT"]]

,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,PAY_OVERDUE_COUNT,WEIGHTED_PAYMENT_HISTORY,AVG_PAY_AMT,SEX_2,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,EDUCATION_5,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,DEFAULT
ID,,,,,,,,,,,,,,,,,,,,
1,24,2,2,-1,-1,-2,-2,2,1.683333,114.833333,1,0,1,0,0,0,1,0,0,1
2,26,-1,2,0,0,0,2,2,0.333333,833.333333,1,0,1,0,0,0,0,1,0,1
3,34,0,0,0,0,0,0,0,0.000000,1836.333333,1,0,1,0,0,0,0,1,0,0
4,37,0,0,0,0,0,0,0,0.000000,1398.000000,1,0,1,0,0,0,1,0,0,0
5,57,-1,0,-1,0,0,0,0,-1.333333,9841.500000,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,39,0,0,0,0,0,0,0,0.000000,7091.666667,0,0,0,1,0,0,1,0,0,0
29997,43,-1,-1,-1,-1,0,0,0,-2.083333,2415.000000,0,0,0,1,0,0,0,1,0,0
29998,37,4,3,2,-1,0,0,3,5.916667,5216.666667,0,0,1,0,0,0,0,1,0,1


In [13]:
credit_cards_df[feature_names.tolist() + ["DEFAULT"]] \
    .to_parquet("../data/credit-cards-reduced.parquet")